In [ ]:
# import tarfile
# import os

# def extract_tar_gz(file_path: str, extract_to: str = "."):
#     """
#     Extracts a .tar.gz file to the specified directory.

#     :param file_path: Path to the .tar.gz file
#     :param extract_to: Directory to extract files to (default: current directory)
#     """
#     if not os.path.exists(file_path):
#         raise FileNotFoundError(f"{file_path} does not exist")

#     # Ensure the extraction directory exists
#     os.makedirs(extract_to, exist_ok=True)

#     with tarfile.open(file_path, "r:gz") as tar:
#         tar.extractall(path=extract_to)
#         print(f"Extracted {file_path} to {extract_to}")


# extract_tar_gz("data/torob-turbo-stage2.tar.gz", "data")


C:\Users\Behnam_Local\AppData\Local\Temp\ipykernel_8380\652266606.py:18: DeprecationWarning: Python 3.14 will, by default, filter extracted tar archives and reject files or modify their metadata. Use the filter argument to control this behavior.
  tar.extractall(path=extract_to)


Extracted data/torob-turbo-stage2.tar.gz to data


In [1]:
import pandas as pd
import os
import glob

# Folder containing Parquet files
data_folder = "data/"

# Get all Parquet files in the folder
parquet_files = glob.glob(os.path.join(data_folder, "*.parquet"))

if not parquet_files:
    raise FileNotFoundError("No Parquet files found in the data folder")

# Iterate through each file
for f in parquet_files:
    print(f"=== File: {os.path.basename(f)} ===")
    
    # Load the Parquet file
    df = pd.read_parquet(f)
    
    # Basic analysis
    print(f"Number of rows: {len(df)}")
    print(f"Number of columns: {len(df.columns)}")
    print(f"Column names: {list(df.columns)}")
    print("Data types:\n", df.dtypes)
    
    # Show first few rows
    print("\nSample rows:")
    print(df.head(), "\n")
    
    print("-" * 50, "\n")


=== File: base_products.parquet ===
Number of rows: 1022298
Number of columns: 8
Column names: ['image_url', 'random_key', 'category_id', 'brand_id', 'english_name', 'persian_name', 'extra_features', 'members']
Data types:
 image_url         object
random_key        object
category_id        int64
brand_id           int64
english_name      object
persian_name      object
extra_features    object
members           object
dtype: object

Sample rows:
                                           image_url random_key  category_id  \
0  https://image.torob.com/base/images/K1/aU/K1aU...     chjknu          338   
1  https://image.torob.com/base/images/vx/gL/vxgL...     ppndkv         2453   
2  https://image.torob.com/base/images/KW/2K/KW2K...     lwtjlj         2300   
3  https://image.torob.com/base/images/VK/R5/VKR5...     ruqoum         1905   
4  https://image.torob.com/base/images/eN/fh/eNfh...     gspcjx          350   

   brand_id       english_name  \
0        -1                      

In [62]:
import psycopg2
from psycopg2 import sql, OperationalError

# Database connection parameters
db_config = {
    "host": "81.12.30.45",
    "port": 29616,
    "dbname": "postgres",
    "user": "postgres",
    "password": "80aEx2MQtiWSlrWmPB16"
}

def create_connection(config):
    """Create a connection to the PostgreSQL database."""
    try:
        connection = psycopg2.connect(**config)
        print("Connection successful")
        return connection
    except OperationalError as e:
        print(f"The error '{e}' occurred")
        return None

def close_connection(connection):
    """Close the database connection."""
    if connection:
        connection.close()
        print("Connection closed")

# Example usage
conn = create_connection(db_config)

Connection successful


In [42]:
import pandas as pd
def read_parquet_file(file_path):
    """Read a Parquet file into a DataFrame."""
    df = pd.read_parquet(file_path)
    print(f"Loaded {len(df)} rows and {len(df.columns)} columns from {file_path}")
    return df
def infer_postgres_column_types(df):
    """Infer PostgreSQL column types based on pandas dtypes."""
    column_defs = []
    for col, dtype in df.dtypes.items():
        if pd.api.types.is_object_dtype(dtype):
            column_type = "TEXT"
        elif pd.api.types.is_integer_dtype(dtype):
            column_type = "BIGINT"
        elif pd.api.types.is_float_dtype(dtype):
            column_type = "FLOAT"
        elif pd.api.types.is_datetime64_any_dtype(dtype):
            column_type = "TIMESTAMP WITH TIME ZONE"
        elif pd.api.types.is_bool_dtype(dtype):
            column_type = "BOOLEAN"
        else:
            column_type = "TEXT"
        column_defs.append(f"{col} {column_type}")
    return column_defs
def create_table(connection, table_name, column_defs):
    """Create a table if it does not exist."""
    cursor = connection.cursor()
    create_table_query = f"CREATE TABLE IF NOT EXISTS {table_name} ({', '.join(column_defs)});"
    cursor.execute(create_table_query)
    connection.commit()
    cursor.close()
    print(f"Table '{table_name}' created or already exists.")
from tqdm import tqdm
def insert_dataframe(connection, df, table_name, batch_size=1000):
    """Insert a pandas DataFrame into PostgreSQL in batches."""
    cursor = connection.cursor()
    cols = df.columns.tolist()
    values_template = ", ".join(["%s"] * len(cols))

    insert_query = sql.SQL("INSERT INTO {} ({}) VALUES ({})").format(
        sql.Identifier(table_name),
        sql.SQL(", ").join(map(sql.Identifier, cols)),
        sql.SQL(values_template)
    )

    data_tuples = [tuple(x) for x in df.to_numpy()]
    total_rows = len(data_tuples)

    for i in tqdm(range(0, total_rows, batch_size)):
        batch = data_tuples[i:i + batch_size]
        cursor.executemany(insert_query, batch)
        connection.commit()
        # print(f"Inserted rows {i}–{i+len(batch)-1} into '{table_name}'")

    cursor.close()
    print(f"✅ Finished inserting {total_rows} rows into '{table_name}'")

In [44]:
pname = "brands"
file_path = f"data/{pname}.parquet"
df = read_parquet_file(file_path)
cols_sql = infer_postgres_column_types(df)
create_table(conn, pname, cols_sql)
insert_dataframe(conn, df, pname, batch_size = 10)

Loaded 2025 rows and 2 columns from data/brands.parquet
Table 'brands' created or already exists.


100%|██████████| 203/203 [01:23<00:00,  2.43it/s]

✅ Finished inserting 2025 rows into 'brands'


In [ ]:
import os
import io
import psycopg2
import pandas as pd


# Mapping pandas dtypes → PostgreSQL
dtype_mapping = {
    "int64": "BIGINT",
    "float64": "DOUBLE PRECISION",
    "bool": "BOOLEAN",
    "datetime64[ns]": "TIMESTAMP",
    "datetime64[ns, UTC]": "TIMESTAMPTZ",
    "object": "TEXT"
}

def map_dtype(dtype):
    """Convert pandas dtype → PostgreSQL dtype"""
    dtype_str = str(dtype)
    return dtype_mapping.get(dtype_str, "TEXT")

# Connect once
conn = psycopg2.connect(**db_config)
cursor = conn.cursor()

data_folder = "data"

for file in os.listdir(data_folder):
    if file.endswith(".parquet"):
        table_name = os.path.splitext(file)[0]  # filename = table name
        file_path = os.path.join(data_folder, file)
        
        if table_name not in ['searches']:
            continue
        print(f"📥 Loading {file_path} → {table_name}")

        # Read parquet file into DataFrame
        df = pd.read_parquet(file_path)

        # --- Step 1: Drop & recreate table with proper types ---
        cursor.execute(f'DROP TABLE IF EXISTS "{table_name}" CASCADE;')
        col_defs = ", ".join([
            f'"{col}" {map_dtype(dtype)}' for col, dtype in zip(df.columns, df.dtypes)
        ])
        cursor.execute(f'CREATE TABLE "{table_name}" ({col_defs});')

        buffer = io.StringIO()
        df.to_csv(buffer, index=False, header=False, sep=",", quoting=1)  # quoting=1 → QUOTE_MINIMAL
        buffer.seek(0)

        # --- Step 3: Bulk load with COPY CSV ---
        cursor.copy_expert(f'COPY "{table_name}" FROM STDIN WITH CSV QUOTE \'"\' DELIMITER \',\'', buffer)
        conn.commit()

        print(f"✅ Inserted {len(df)} rows into {table_name}")

cursor.close()
conn.close()
print("🎉 All parquet files loaded successfully with correct types!")


📥 Loading data\searches.parquet → searches


In [ ]:
conn = psycopg2.connect(**db_config)
query = "SELECT * FROM searches;"
df = pd.read_sql(query, conn)

C:\Users\Behnam_Local\AppData\Local\Temp\ipykernel_7340\3930754043.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


In [ ]:
df

In [ ]:
# Remember to close the connection when done
close_connection(conn)